In [ ]:
%matplotlib qt5
import numpy as np
from interactive_image_analysis import InteractiveImageAnalysis

font_path = "/home/somar/.fonts/SourceSansPro-Semibold.otf" 

stacks = ['/home/somar/Desktop/2025/Data for publication/Multilayer graphene/Sample 2476/stacktest1.h5']
stacks_ssb = ['/home/somar/Desktop/2025/Data for publication/Sample 2525/SSB reconstruction of 4d STEM data/stack_ssbs.h5']
stacks_ssb1 = ['/home/somar/Desktop/2025/Data for publication/Sample 2525/SSB reconstruction of 4d STEM data/stack.h5']

stacks_adf = ['/home/somar/Desktop/2025/Data for publication/Sample 2525/ADF images/stack.h5']

# stack = stacks[0]
# s = '/home/somar/test /stack.h5'
# image_analysis = InteractiveImageAnalysis(stacks_ssb1[0], metadata_path=None, analysing_features=False, save_images_with_calibrated_scalebar=TabError, fixed_length_scalebar=True,
#                                            clean_graphene_analysis=True, contamination_analysis=False, clusters_analysis=True , defects_analysis=False, font_path=font_path)

In [ ]:
# from fft_calibration_class import FFTCalibration
# from data_loader import DataLoader
# img_stack = DataLoader(stacks[0])
# metadata = img_stack.raw_metadata

# calibrator = FFTCalibration(stack=img_stack)
# calibrator.fft_calibrate()

In [ ]:
#inch to cm conversion factor
inch_to_cm = 2.54
fig = (16,8) # inches
dpi = 200
img_min = 8 * 200
img_min

In [ ]:
font_path = "/home/somar/.fonts/SourceSansPro-Semibold.otf" 

from calibrated_images_class import CalibratedImages
calibrated_images = CalibratedImages(stacks[0], font_path=font_path)